In [1]:
import numpy as np
import re
import torch
from tqdm.notebook import tqdm
from torch.optim import AdamW

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 89.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 92.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 20.9 MB/s eta 0:00:00


In [9]:
import transformers

In [3]:
from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('sberbank-ai/rugpt3small_based_on_gpt2')

In [4]:
with open('dune.txt', encoding='utf-8') as f:
    text = f.read()

text = re.sub('\n{2,}', '\n', text)
print(text[:1000])

Начало есть время, когда следует позаботиться о том, чтобы все было отмерено и уравновешено. Это знает каждая сестра Бене Гессерит, Итак, приступая к изучению жизни Муад'Диба, прежде всего правильно представьте время его: рожден в пятьдесят седьмой год правления Падишах-Императора Шаддама IV. И с особым вниманием отнеситесь к его месту в пространстве: планете Арракис. Пусть не смутит вас то, что родился он на Каладане и первые пятнадцать лет своей жизни провел на этой планете: Арракис, часто называемой также Дюной, — вот место Муад'Диба, вовеки.
Из учебника «Жизнь Муад'Диба» принцессы Ирулан
За неделю до отлета на Арракис, когда суета приготовлений и сборов достигла апогея, превратившись в настоящее безумие, какая-то сморщенная старуха пришла к матери Пауля.
Над замком Каладан стояла теплая ночь, но из древних каменных стен, двадцать шесть поколений служивших роду Атрейдесов, как всегда перед сменой погоды, выступил тонкий, прохладный налет влаги.
Старуху впустили через боковую дверь, 

In [5]:
tokens = tokenizer.encode(text, add_special_tokens=True)
tokens = np.array(tokens)
print(len(tokens))
tokens[:10]

193298


array([ 2176, 11882,   912,   808,    16,   785,  3014, 31675,   290,
         970])

In [6]:
l = len(tokens)//15
train = []
test = []
for i in range(15):
    if i%5 > 0:
        train.extend(tokens[i*l: (i+1)*l])
    else:
        test.extend(tokens[i*l: (i+1)*l])
train = np.array(train)
test = np.array(test)

print(len(tokens), len(train), len(test))

193298 154632 38658


In [7]:
from transformers import GPT2LMHeadModel
model = GPT2LMHeadModel.from_pretrained(
    'sberbank-ai/rugpt3small_based_on_gpt2',
    output_attentions = False,
    output_hidden_states = False,
)

model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50264, 768)
    (wpe): Embedding(2048, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

In [10]:
batch_size = 8
max_len = 128
epochs = 5

n_train = len(train)//(batch_size*max_len)
n_test = len(test)//(batch_size*max_len)
print(n_train, n_test)

optimizer = AdamW(model.parameters(), lr = 1e-5, eps = 1e-8)

total_steps = n_train * epochs
scheduler = transformers.get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

151 37


In [27]:
def prep_tensors(x, i, batch_size=batch_size, max_len=max_len):
    batch_ids = x[i*batch_size*max_len: (i+1)*batch_size*max_len]
    batch_ids = batch_ids.reshape(batch_size, max_len)
    batch_ids = torch.tensor(batch_ids).to(device)
    return batch_ids

for epoch in range(1, epochs+1):
    print(f'epoch {epoch}/{epochs} : training')
    train_loss = []

    model.train()
    pbar = tqdm(range(n_train))
    for i in pbar:
        batch_ids = prep_tensors(train, i)

        model.zero_grad()
        loss, logits, _ = model(batch_ids,
                                token_type_ids=None, 
                                labels=batch_ids
                             ).values()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        
        train_loss.append(loss.item())
        pbar.set_description(f'loss {np.mean(train_loss):.4f}', refresh=True)

    
    print(f'epoch {epoch}/{epochs} : validation')
    model.eval()
    val_loss = []
    pbar = tqdm(range(n_test))
    for i in pbar:
        batch_ids = prep_tensors(test, i)
        with torch.no_grad():        
            loss, logits, _ = model(batch_ids, 
                                token_type_ids=None,
                                labels=batch_ids
                                 ).values()
        
        val_loss.append(loss.item())
        pbar.set_description(f'loss {np.mean(val_loss):.4f}', refresh=True)


epoch 1/5 : training


  0%|          | 0/151 [00:00<?, ?it/s]

epoch 1/5 : validation


  0%|          | 0/37 [00:00<?, ?it/s]

epoch 2/5 : training


...

  0%|          | 0/37 [00:00<?, ?it/s]

epoch 5/5 : training


  0%|          | 0/151 [00:00<?, ?it/s]

epoch 5/5 : validation


  0%|          | 0/37 [00:00<?, ?it/s]

In [28]:
torch.save(model.state_dict(), 'trained_on_dune.pt')

In [29]:
import textwrap

In [32]:
prompt = 'Что нужно взять с собой в дорогу'
prompt = tokenizer.encode(prompt, return_tensors='pt').to(device)
out = model.generate(
    input_ids=prompt,
    max_length=150,
    num_beams=5,
    do_sample=True,
    temperature=5.0,
    top_k=50,
    top_p=0.6,
    no_repeat_ngram_size=2,
    num_return_sequences=1,
    ).cpu().numpy()
for out_ in out:
    print(textwrap.fill(tokenizer.decode(out_), 100), end='\n------------------\n')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Что нужно взять с собой в дорогу Эмм… Мне нужно купить для отца оружие – не считая ножей для охоты и
оружия для пикета. Он считает себя достаточно умным и способным разбираться в людях – но все же не
способен понимать искусство и искусство воина! А вот что, мой милый мальчик: для победы в бою нужны
мужество и хитрость – а хитрость не знает границ, она требует ума и ума! Но все это лишь кажущаяся
подготовка! Настоящую подготовку надо готовить, тренируя каждую свою… женщину! У нее нет выбора:
либо погибнуть от стрелы-дракулы в сражении или же быть убитым или оказаться рабом-скрипуче и
неуклюже управляющим ею. Это одно из проявлений силы духа женщины!.. Если бы ее научи
------------------
